In [23]:
import os
from PIL import Image
import pandas as pd 
import glob
import random as r
import tqdm
import json

In [2]:
loc = "../data/"
labels = glob.glob(loc + "**/**/Labeled*/**/*.json")

In [63]:
labels

['../data/rockfish_full/SHE-11/Labeled_8_s_1_100/export/8_stbd_1_100_Joy.json',
 '../data/rockfish_full/SHE-11/Labeled_21_s_1_100/export_1_100/export-2020-12-05T09_49_04.355Z.json',
 '../data/rockfish_full/SHE-11/Labeled_7_s_101_150/export/export-2020-12-06T06_31_45.283Z.json',
 '../data/rockfish_full/SHE-11/Labeled_3_p_1_100/export/3_port_1_100.json',
 '../data/rockfish_full/SHE-11/Labeled_17_p_1_100/export/export-2020-12-06T07_05_18.687Z.json',
 '../data/rockfish_full/SHE-11/Labeled_11_S_1_100/export/export-2020-12-06T06_37_13.628Z.json',
 '../data/rockfish_full/SHE-11/Labeled_7_s_1_100/export/export-2020-12-06T06_31_32.231Z.json']

In [71]:
# I want the dirs truncated at "Labeled_..."
# not sure what the os command is to do what I'm doing here 

for l in labels:
    split_dirs = os.path.dirname(l).split("/")
    split_dirs = split_dirs[2:-1]
    rejoin = "/".join(split_dirs)
    print(rejoin)

rockfish_full/SHE-11/Labeled_8_s_1_100
rockfish_full/SHE-11/Labeled_21_s_1_100
rockfish_full/SHE-11/Labeled_7_s_101_150
rockfish_full/SHE-11/Labeled_3_p_1_100
rockfish_full/SHE-11/Labeled_17_p_1_100
rockfish_full/SHE-11/Labeled_11_S_1_100
rockfish_full/SHE-11/Labeled_7_s_1_100


In [4]:
total_ims = 0
total_bboxes = 0
for l in labels:
    label_list = json.load(open(l, "r"))
    total_ims += len(label_list)
    for im in label_list:
        bboxes = im['Label']['objects']
        total_bboxes += len(bboxes)
        
print(total_ims, "Total images")
print(total_bboxes, "Total bboxes")

650 Total images
15128 Total bboxes


In [5]:
label_list[0]['External ID'] # this is the image name

'20111010.172736.00062_uncropped_rect_color.jpg'

In [72]:
def get_image_path(labels_path):
    """
    Takes path to labels and truncates it to path to images.
    I need this to append image paths to each image file.
    """
    split_lp = os.path.dirname(labels_path).split("/")
    split_lp = split_lp[2:-1]
    rejoined = "/".join(split_lp)
    
    return rejoined

In [73]:
box_list = []
for l in labels:
    images_path = get_image_path(l)
    label_list = json.load(open(l, "r"))
    for _, label in enumerate(label_list):
        filename = label['External ID']
        bboxes = label['Label']['objects']
        for box in bboxes:
            x_min = box['bbox']['left']
            y_min = box['bbox']['top']
            x_max = x_min + box['bbox']['width']
            y_max = y_min + box['bbox']['height']
            box_list.append({
                "x_min": x_min,
                "x_max": x_max,
                "y_min": y_min,
                "y_max": y_max,
                "filename": os.path.join(images_path, filename),
                "title": box['title'],
                "value": box['value']
            })
        
jimmy_df = pd.DataFrame(box_list)

In [36]:
jimmy_df.title.value_counts()

Animal                10114
animal                 4926
No Animal                44
no animal                27
No Visible Animals       17
Name: title, dtype: int64

In [74]:
jimmy_df

,x_min,x_max,y_min,y_max,filename,title,value
0,702,1380,539,1041,rockfish_full/SHE-11/Labeled_8_s_1_100/2011101...,no animal,no_animal
1,1166,1298,394,512,rockfish_full/SHE-11/Labeled_8_s_1_100/2011101...,animal,animal
2,551,704,1189,1347,rockfish_full/SHE-11/Labeled_8_s_1_100/2011101...,animal,animal
3,1327,1451,1086,1250,rockfish_full/SHE-11/Labeled_8_s_1_100/2011101...,animal,animal
4,1087,1180,782,877,rockfish_full/SHE-11/Labeled_8_s_1_100/2011101...,animal,animal
...,...,...,...,...,...,...,...
15123,1713,1874,1536,1631,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15124,1643,1826,1297,1433,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15125,1304,1427,665,765,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal
15126,737,943,221,346,rockfish_full/SHE-11/Labeled_7_s_1_100/2011101...,Animal,animal


In [39]:
jimmy_df.value.value_counts()

animal                15040
no_animal                71
no_visible_animals       17
Name: value, dtype: int64

In [75]:
jimmy_df.to_csv("jimmy_labels.csv", index=False)